In [1]:
import polars as pl
import os

In [2]:
# Paths
IMDB_PATH = '../data/raw/imdb'
MOVIELENS_PATH = '../data/raw/movie-lens'
TMDB_PATH = '../data/raw/tmdb'

In [ ]:
# IMDb Files
imdb_files = {
    "basics": "title.basics.tsv",
    "akas": "title.akas.tsv",
    "crew": "title.crew.tsv",
    "episode": "title.episode.tsv",
    "principals": "title.principals.tsv",
    "ratings": "title.ratings.tsv",
    "names": "name.basics.tsv"
}

# Load IMDb Data (Polars primary, Pandas fallback for messy files)
imdb_dfs = {}

for key, filename in imdb_files.items():
    path = os.path.join(IMDB_PATH, filename)
    try:
        imdb_dfs[key] = pl.read_csv(
            path, 
            separator='\t', 
            null_values='\\N', 
            infer_schema_length=1000,
            quote_char=None  # This avoids quote parsing issues
        )
    except pl.ComputeError as e:
        print(f"⚠️ Polars failed reading {key}, falling back to pandas")
        imdb_dfs[key] = pl.from_pandas(pd.read_csv(path, sep='\t', na_values='\\N'))  # Convert to Polars for consistency

# MovieLens Files
movielens_files = {
    "movies": "movies.csv",
    "ratings": "ratings.csv",
    "tags": "tags.csv",
    "links": "links.csv",
    "genome_scores": "genome_scores.csv",
    "genome_tags": "genome_tags.csv"
}

movielens_dfs = {
    key: pl.read_csv(os.path.join(MOVIELENS_PATH, filename))
    for key, filename in movielens_files.items()
}

# TMDb File
tmdb_movies = pl.read_csv(os.path.join(TMDB_PATH, 'tmdb_2023.csv'))

# Quick Check
for name, df in imdb_dfs.items():
    print(f"IMDb - {name}:")
    print(df.head())

for name, df in movielens_dfs.items():
    print(f"MovieLens - {name}:")
    print(df.head())

print("TMDb:")
print(tmdb_movies.head())

ComputeError: could not parse `"Rome brûle" (Portrait de Shirley Clarke)` as dtype `str` at column 'primaryTitle' (column number 3)

The current offset in the file is 3679531 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `"Rome brûle" (Portrait de Shirley Clarke)` to the `null_values` list.

Original error: ```invalid csv file

Field `"Rome brûle" (Portrait de Shirley Clarke)` is not properly escaped.```